Сравним время операций **insert** и **append** для массива.

In [1]:
power = 17 # change me

In [2]:
a = list(range(2**power))

In [3]:
len(a)

131072

In [4]:
%%timeit
a.insert(1, 1)

85.2 µs ± 9.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [5]:
a = list(range(2**power))

In [6]:
%%timeit
a.append(1)

31.5 ns ± 0.151 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [7]:
len(a)

81242183

In [8]:
%%timeit
len(a)

37.9 ns ± 0.212 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


Посмотрим, как в питоне происходит динамическое расширение массива. Воспользуемся функцией **getsizeof** для получение размера объекта в байтах.

In [9]:
from sys import getsizeof

In [10]:
a = []

In [11]:
getsizeof(a)

56

In [12]:
a = []
for i in range(10):
    print(i, getsizeof(a))
    a.append(i)



0 56
1 88
2 88
3 88
4 88
5 120
6 120
7 120
8 120
9 184


In [13]:
# []
# [0 _ _ _]
# [0 1 2 3]
# [0 1 2 3 4 _ _ _]

In [14]:
def getsizeof2(a):
    return getsizeof(a) - 56

In [15]:
a = []
capacity = getsizeof2(a)
# добавляем элементы в массив и смотрим, когда и во сколько раз он расширился
for i in range(1000000):
    a.append(i)
    if getsizeof2(a) != capacity:
        if capacity != 0:
            print(i, getsizeof2(a) / capacity)
        capacity = getsizeof2(a)


4 2.0
8 2.0
16 1.5
24 1.3333333333333333
32 1.25
40 1.3
52 1.2307692307692308
64 1.1875
76 1.2105263157894737
92 1.173913043478261
108 1.1851851851851851
128 1.15625
148 1.162162162162162
172 1.1627906976744187
200 1.16
232 1.1551724137931034
268 1.1492537313432836
308 1.1428571428571428
352 1.1363636363636365
400 1.14
456 1.1403508771929824
520 1.1384615384615384
592 1.135135135135135
672 1.130952380952381
760 1.131578947368421
860 1.130232558139535
972 1.131687242798354
1100 1.1309090909090909
1244 1.1286173633440515
1404 1.1282051282051282
1584 1.128787878787879
1788 1.1275167785234899
2016 1.126984126984127
2272 1.1267605633802817
2560 1.1265625
2884 1.1262135922330097
3248 1.1268472906403941
3660 1.126775956284153
4124 1.1260911736178467
4644 1.1257536606373815
5228 1.1262433052792655
5888 1.125679347826087
6628 1.1255280627640314
7460 1.1254691689008043
8396 1.1257741781800858
9452 1.125687685146001
10640 1.125563909774436
11976 1.1255845023380093
13480 1.1255192878338278
15172 1

Видим, что расширение происход при полном заполнении массива, и для уже начиная с размера **~2000** массивы расшираются в **1.125** раз, то есть на одну восьмую

Сужение происходит реже, примерно когда в массиве осталось **9/16**, или **0.5625** заполненных элементов.

### Linked List vs Array

In [16]:
# Так можно реализовать связный список в Python. В качестве упражнения можно добавить методы удаления, вставки в середину.

class Node:

    def __init__(self, nxt, value):
        self.nxt = nxt
        self.value = value


class LinkedList:

    def __init__(self):
        self.head = None
        self.tail = None

    def append(self, value):
        if self.head is None:
            self.head = Node(nxt=None, value=value)
            self.tail = self.head
        else:
            self.tail.nxt = Node(nxt=None, value=value)
            self.tail = self.tail.nxt

    def __getitem__(self, item):
        cur = self.head
        i = 0
        while i != item:
            cur = cur.nxt
            i += 1

        return cur.value

In [17]:
def populate_array(size: int): 
    new = list()
    for i in range(size):
        new.append(i)
        
    return new

def populate_list(size: int): 
    new = LinkedList()
    for i in range(size):
        new.append(i)
        
    return new



In [18]:
import time

def timing(func, *args, n_repeats: int = 20):
    timings = []
    for _ in range(n_repeats):
        start = time.time()
        x = func(*args)
        timings.append(time.time() - start)
        
    return x, sum(timings) / len(timings)
        

In [19]:
def timeit_doubling(func, max_power: int = 17, n_repeats: int = 20):
    prev = 1
    for power in range(2, max_power, 1):
        _, curr = timing(func, 2**power)
        ratio = curr / prev
        print(2**power, ',', 'ratio:',  round(ratio, 3), 'time:', round(curr, 3))
        prev = curr
    

Измерим время, которое требуется на создание массива и списка из N элементов, и посмотрим, как оно меняется при увеличении N

In [20]:
timeit_doubling(populate_array)

4 , ratio: 0.0 time: 0.0
8 , ratio: 0.792 time: 0.0
16 , ratio: 1.429 time: 0.0
32 , ratio: 1.833 time: 0.0
64 , ratio: 1.7 time: 0.0
128 , ratio: 1.995 time: 0.0
256 , ratio: 1.887 time: 0.0
512 , ratio: 2.114 time: 0.0
1024 , ratio: 1.931 time: 0.0
2048 , ratio: 2.017 time: 0.0
4096 , ratio: 1.978 time: 0.0
8192 , ratio: 1.986 time: 0.0
16384 , ratio: 1.991 time: 0.001
32768 , ratio: 1.976 time: 0.001
65536 , ratio: 2.04 time: 0.002


In [21]:
timeit_doubling(populate_list)

4 , ratio: 0.0 time: 0.0
8 , ratio: 1.822 time: 0.0
16 , ratio: 1.92 time: 0.0
32 , ratio: 1.916 time: 0.0
64 , ratio: 1.963 time: 0.0
128 , ratio: 2.074 time: 0.0
256 , ratio: 1.974 time: 0.0
512 , ratio: 2.289 time: 0.0
1024 , ratio: 5.622 time: 0.001
2048 , ratio: 0.759 time: 0.001
4096 , ratio: 3.463 time: 0.003
8192 , ratio: 1.851 time: 0.006
16384 , ratio: 2.2 time: 0.014
32768 , ratio: 2.357 time: 0.033
65536 , ratio: 2.206 time: 0.072


В обоих случаях это время линейно, но в абсолютных числах создание списка существенно медленней.

### Длинная арифметика в Python

In [22]:
a, b = 200, 300

In [23]:
%%timeit
a * b

30.8 ns ± 0.361 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [24]:
a, b = 2**25, 2**25

In [25]:
%%timeit
a * b

30.8 ns ± 0.276 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [26]:
a, b = 2**200, 2**200

In [27]:
%%timeit
a * b

156 ns ± 1.24 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Стек

In [28]:
class Stack:
    
    def __init__(self):
        self.stack = []
        
    def push(self, value: int):
        self.stack.append(value)
    
    def peek(self) -> int:
        if self.stack:
            return self.stack[-1]
        
        return None
    
    def pop(self) -> int:
        value = None
        if self.stack:
            value = self.stack[-1]
            self.stack = self.stack[:-1] # O(N)
            
        return value
    
    def pop_fast(self) -> int:
        pass

In [30]:
a = Stack()

In [31]:
for i in range(10**5):
    a.push(i)

In [32]:
for i in range(10**5):
    a.pop()

In [33]:
for i in range(10**5):
    a.pop_fast()

In [34]:
a.stack

[]